In [13]:
import numpy
import tensorflow as tf
import os

from tensorflow.python.platform import gfile

def _read32(bytestream):
    dt = numpy.dtype(numpy.uint32).newbyteorder('>')
    return int(numpy.frombuffer(bytestream.read(4), dtype=dt)[0])

def _dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = numpy.arange(num_labels) * num_classes
    labels_one_hot = numpy.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    labels_one_hot = labels_one_hot.astype(numpy.float32)
    return labels_one_hot

def input_mnist_data(file_path):
    with gfile.Open(file_path, 'rb') as bytestream:
        magic = _read32(bytestream)
        if magic != 2051:
            raise ValueError('Invalid magic number %d in MNIST image file: %s' % (magic, file_path))

        num_images = _read32(bytestream)
        rows = _read32(bytestream)
        cols = _read32(bytestream)
        buf = bytestream.read(rows * cols * num_images)
        data = numpy.frombuffer(buf, dtype=numpy.uint8)
        data = data.reshape(num_images, rows * cols * 1)
        data = data.astype(numpy.float32)
        return data
    
def input_mnist_label(file_path):    
    with gfile.Open(file_path, 'rb') as bytestream:
        magic = _read32(bytestream)
        if magic != 2049:
            raise ValueError( 'Invalid magic number %d in MNIST label file: %s' % (magic, file_path) )
        
        num_items = _read32(bytestream)
        buf = bytestream.read(num_items)
        labels = numpy.frombuffer(buf, dtype=numpy.uint8)
        return _dense_to_one_hot(labels, 10)


def _get_next_data(image_file, label_file, batch_size):
    train_data = input_mnist_data(image_file)
    train_label = input_mnist_label(label_file)
    step = 0
    for step in range( len(train_data) // batch_size ):
        begin = step * batch_size
        end = begin + batch_size
        if end >= len(train_data):
            end = len(train_data) - 1

        yield train_data[begin : end], train_label[begin : end]


def inference(features):
    combine_input = tf.matmul(features, _W) + _b
    inference = tf.nn.softmax( combine_input )      
    inference = tf.Print(inference, [inference, combine_input], '******* inference: ')
    return inference
    
print( '-------------- do_train: start -----------------' )


_W = tf.Variable(tf.zeros([784, 10]), name = 'weights')
_b = tf.Variable(tf.zeros([10]), name = 'bias')    

print( '0 -W: ', _W )
       
train_data_generator = _get_next_data( './MNIST_data/train-images.idx3-ubyte', 
                                      './MNIST_data/train-labels.idx1-ubyte', 5 )

# features, labels = next( data_generator )
# print( '0 label: ', labels )
features = tf.placeholder("float", shape=[None, 784])
labels = tf.placeholder("float", shape=[None, 10])

combine_input = tf.matmul(features, _W) + _b
combine_input = tf.Print(combine_input, [combine_input], '0 ****************** combine_input: ')
label_predicted = tf.nn.softmax( combine_input )
cost = -tf.reduce_mean( labels * tf.log( label_predicted ) )

#label_predicted = inference( features )
#print( '0 label_predicted: ', label_predicted )



#cost = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits(logits = combine_input, labels = labels) )
cost = tf.Print(cost, [cost], '0 ****************** cost: ')

with tf.control_dependencies([tf.Print(cost, [cost], '0 ****************** cost: ')]):
    train_step = tf.train.GradientDescentOptimizer(0.01).minimize( cost )


sess = tf.InteractiveSession()
sess.run( tf.initialize_all_variables() )


print( '1 _W: ', sess.run(_W) )
print( '1_0 cost: ', cost )
# print( '1_1 cost: ', sess.run(cost) )

#print( '1_1 label_predicted: ', sess.run(label_predicted) )
#print( '1_1 labels_t: ', sess.run(labels_t) )
for i in range(10):
    features_in, labels_in = next( train_data_generator )
    print( '11_{} features_in: '.format(i), features_in )
    print( '11_{} labels_in: '.format(i), labels_in )
    
    sess.run( train_step, feed_dict={features: features_in, labels: labels_in} )
    print( '2_1 cost: ', cost.eval(feed_dict={features: features_in, labels: labels_in}) )
        
print( '2_0 cost: ', cost )
print( '2 _W: ', sess.run(_W) )
    

correct_prediction = tf.equal(tf.argmax(label_predicted, 1), tf.argmax(labels, 1))
#print( '3 correct_prediction: ', sess.run(correct_prediction) )

accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))   
#print( '3 accuracy: ', sess.run(accuracy) )    

test_data_generator = _get_next_data( './MNIST_data/t10k-images.idx3-ubyte', 
                                      './MNIST_data/t10k-labels.idx1-ubyte', 5 )
test_features, test_labels = next( test_data_generator )
print( accuracy.eval(feed_dict={features: test_features, labels: test_labels}) )
    
 
    


-------------- do_train: start -----------------
0 -W:  <tf.Variable 'weights_8:0' shape=(784, 10) dtype=float32_ref>
Instructions for updating:
Use `tf.global_variables_initializer` instead.
1 _W:  [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
1_0 cost:  Tensor("Print_15:0", shape=(), dtype=float32)
11_0 features_in:  [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
11_0 labels_in:  [[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]
2_1 cost:  nan
11_1 features_in:  [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 

In [37]:
import tensorflow as tf

features_in = [[ 0.,  0.,  0., 255., 123., 0., 234., 111., 9.,  0.,  0.,  0.],
 [ 0.,  0.,  0., 2., 13., 20., 24., 11., 39.,  50.,  0.,  0.],
 [ 0.,  0.,  0., 25., 12., 30., 34., 111., 93.,  0.,  0.,  0.]]
    
labels_in = [[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
 [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]
    
    
_W = [[ 0.2,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]

_b = [ 0.2,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.1]


logits = tf.matmul(features_in, _W) + _b
logits = tf.Print(logits, [logits], '0 logits: ')


label_predicted = tf.nn.softmax(logits)
label_predicted = tf.Print(label_predicted, [label_predicted], '0 label_predicted: ')
cost1_0 = labels_in * tf.log(label_predicted)
cost1 = -tf.reduce_mean( cost1_0 )

#################### fuck! #######################
# cost2 = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels_in) )
cost2_0 = tf.nn.softmax_cross_entropy_with_logits(logits, labels_in)
cost2 = tf.reduce_mean( cost2_0 )


sess = tf.InteractiveSession()

with tf.control_dependencies([tf.Print(cost1, [cost1], "###")]):
    print('logits: ', sess.run(logits))
    print('softmax result=', sess.run(label_predicted))
    print('cost1_0 =', sess.run(cost1_0))
    print('cost2_0 =', sess.run(cost2_0))
    c_1 = sess.run(cost1)
    c_2 = sess.run(cost2)
    print(c_1)
    print(c_2)

logits:  [[ 0.2  0.   0.   0.   0.   0.   0.   0.   0.   0.1]
 [ 0.2  0.   0.   0.   0.   0.   0.   0.   0.   0.1]
 [ 0.2  0.   0.   0.   0.   0.   0.   0.   0.   0.1]]
softmax result= [[ 0.11827764  0.09683754  0.09683754  0.09683754  0.09683754  0.09683754
   0.09683754  0.09683754  0.09683754  0.10702203]
 [ 0.11827764  0.09683754  0.09683754  0.09683754  0.09683754  0.09683754
   0.09683754  0.09683754  0.09683754  0.10702203]
 [ 0.11827764  0.09683754  0.09683754  0.09683754  0.09683754  0.09683754
   0.09683754  0.09683754  0.09683754  0.10702203]]
cost1_0 = [[-0.         -0.         -0.         -0.         -2.33472061 -0.         -0.
  -0.         -0.         -0.        ]
 [-2.13472056 -0.         -0.         -0.         -0.         -0.         -0.
  -0.         -0.         -0.        ]
 [-0.         -2.33472061 -0.         -0.         -0.         -0.         -0.
  -0.         -0.         -0.        ]]
cost2_0 = [ 2.33472061  2.13472056  2.33472061]
0.226805
2.26805


Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f4a18616d68>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 171, in __del__
    self.close()
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 976, in close
    self._default_session.__exit__(None, None, None)
  File "/usr/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/ops.py", line 3378, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects


In [34]:
import tensorflow as tf  

#our NN's output  
#logits=tf.constant([[1.0,2.0,3.0, 4.0],[1.0,2.0,3.0,4.0],[1.0,2.0,3.0,4.0]])  
logits = [[0.2, 0., 0., 0., 0., 0., 0., 0., 0., 0.1],
 [0.2, 0., 0., 0., 0., 0., 0., 0., 0., 0.1],
 [0.2, 0., 0., 0., 0., 0., 0., 0., 0., 0.1]]
#step1:do softmax  
y=tf.nn.softmax(logits)  
#true label  
#y_=tf.constant([[0.0,0.0,0.0,1.0],[0.0,0.0,0.0,1.0],[0.0,0.0,0.0,1.0]])  
y_ = [[ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
 [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
 [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]
#step2:do cross_entropy  
cross_entropy = -tf.reduce_sum(y_*tf.log(y))  
#do cross_entropy just one step  
cross_entropy2=tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits, y_))#dont forget tf.reduce_sum()!!  

with tf.Session() as sess:  
    softmax=sess.run(y)  
    c_e = sess.run(cross_entropy)  
    c_e2 = sess.run(cross_entropy2)  
    print("step1:softmax result=")  
    print(softmax)  
    print("step2:cross_entropy result=")  
    print(c_e)  
    print("Function(softmax_cross_entropy_with_logits) result=")  
    print(c_e2)

step1:softmax result=
[[ 0.11827764  0.09683754  0.09683754  0.09683754  0.09683754  0.09683754
   0.09683754  0.09683754  0.09683754  0.10702203]
 [ 0.11827764  0.09683754  0.09683754  0.09683754  0.09683754  0.09683754
   0.09683754  0.09683754  0.09683754  0.10702203]
 [ 0.11827764  0.09683754  0.09683754  0.09683754  0.09683754  0.09683754
   0.09683754  0.09683754  0.09683754  0.10702203]]
step2:cross_entropy result=
6.80416
Function(softmax_cross_entropy_with_logits) result=
6.80416


Exception ignored in: <bound method BaseSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f4a1aead0b8>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 171, in __del__
    self.close()
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py", line 976, in close
    self._default_session.__exit__(None, None, None)
  File "/usr/lib/python3.5/contextlib.py", line 66, in __exit__
    next(self.gen)
  File "/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/ops.py", line 3378, in get_controller
    % type(default))
AssertionError: Nesting violated for default stack of <class 'weakref'> objects
